In [1]:
!pip install xgboost

In [2]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier


In [3]:
# Load your dataset
df = pd.read_csv("job_title_des.csv")

# Drop rows where job description is missing
df.dropna(subset=['Job Description'], inplace=True)

In [4]:
# Define skill keywords to extract
skill_keywords = ['python', 'sql', 'excel', 'power bi', 'machine learning', 'tensorflow',
                  'r', 'tableau', 'communication', 'statistics', 'numpy', 'pandas', 'spark']

# Function to extract matched skills
def extract_skills(text):
    text = text.lower()
    return list(set(skill for skill in skill_keywords if skill in text))

# Apply extraction
df['Extracted_Skills'] = df['Job Description'].apply(extract_skills)

# Keep only rows with at least one skill
df = df[df['Extracted_Skills'].map(len) > 0]


In [5]:
# TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=1500)
X = tfidf.fit_transform(df['Job Description'])

# Label Binarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Extracted_Skills'])


In [6]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model using OneVsRestClassifier with XGBoost
model = OneVsRestClassifier(XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
model.fit(X_train, y_train)


C:\Users\manik\anaconda3\lib\site-packages\xgboost\training.py:183: UserWarning: [21:49:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\manik\anaconda3\lib\site-packages\xgboost\training.py:183: UserWarning: [21:49:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\manik\anaconda3\lib\site-packages\xgboost\training.py:183: UserWarning: [21:49:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\manik\anaconda3\lib\site-packages\xgboost\training.py:183: UserWarning: [21:49:13] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtr

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            feature_types=None,
                                            feature_weights=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None, ...))

In [7]:
# Example input (you can replace this with your own job description later)
new_job_description = ["Looking for an expert in Python, SQL, Tableau, and strong communication skills."]

# Transform input using the same TF-IDF vectorizer
X_new = tfidf.transform(new_job_description)

# Predict and decode labels
predicted = model.predict(X_new)
predicted_skills = mlb.inverse_transform(predicted)

# Output
print("✅ Predicted Skills:", predicted_skills[0])


✅ Predicted Skills: ('communication', 'python', 'r', 'sql')


In [8]:
# Different job description input
new_job_description = ["We are hiring a Business Intelligence Analyst proficient in Power BI, Excel, data modeling, and statistical analysis. Strong communication skills are a must."]

# Transform the input using the same TF-IDF vectorizer
X_new = tfidf.transform(new_job_description)

# Predict the skills
predicted = model.predict(X_new)
predicted_skills = mlb.inverse_transform(predicted)

# Output the results
print("✅ Predicted Skills:", predicted_skills[0])


✅ Predicted Skills: ('communication', 'r')


In [9]:
# Different job description input
new_job_description = ["We're hiring a Machine Learning Engineer skilled in Python, TensorFlow, Pandas, and data visualization tools like Power BI or Tableau. Strong understanding of statistics and communication skills are a plus."]
# Transform the input using the same TF-IDF vectorizer
X_new = tfidf.transform(new_job_description)

# Predict the skills
predicted = model.predict(X_new)
predicted_skills = mlb.inverse_transform(predicted)

# Output the results
print("✅ Predicted Skills:", predicted_skills[0])

✅ Predicted Skills: ('communication', 'machine learning', 'python', 'r', 'statistics', 'tensorflow')
